In [1]:
import os
from topic_gen.input.inject_data import InjectData
import pandas as pd
from dotenv import load_dotenv

load_dotenv()


inject = InjectData()

user_id = "8c525b6b-27c9-4379-a454-2c3b3a781124"

df = inject.query_bookmarks(user_id)

print(df)

                               bookmark_id  \
0     a4f37ecc-172a-48e4-8570-8d059398d77d   
1     a4f37ecc-172a-48e4-8570-8d059398d77d   
2     a4f37ecc-172a-48e4-8570-8d059398d77d   
3     06ba62ab-870f-432a-87a6-a5be07f7b2e6   
4     06ba62ab-870f-432a-87a6-a5be07f7b2e6   
...                                    ...   
2132  d8ae06ab-0b62-4115-86c8-7153834e142a   
2133  d8ae06ab-0b62-4115-86c8-7153834e142a   
2134  d8ae06ab-0b62-4115-86c8-7153834e142a   
2135  d8ae06ab-0b62-4115-86c8-7153834e142a   
2136  d8ae06ab-0b62-4115-86c8-7153834e142a   

                                                    url  \
0                https://www.latent.space/p/2025-papers   
1                https://www.latent.space/p/2025-papers   
2                https://www.latent.space/p/2025-papers   
3     https://www.interconnects.ai/p/papers-im-readi...   
4     https://www.interconnects.ai/p/papers-im-readi...   
...                                                 ...   
2132                   https://arx

In [2]:
grouped_df = (
    df.groupby(["tag_id", "key", "name"]).apply(lambda g: g[[
        "bookmark_id", "title", "description", "language", "created_at", "updated_at",
    ]].to_dict(orient="records"))
    .reset_index(name="bookmarks")
)

print(grouped_df)

                                   tag_id          key         name  \
0    00608d78-5556-4e9a-8b0c-ce2796f9f13d         qwen         qwen   
1    006adf0c-c6bc-44b3-90f6-cc9f524a90e0  challenging  challenging   
2    03b8761c-7c70-42d5-bf1a-e04aa63c36b5      testing      testing   
3    07db4626-c83f-4ac5-86d4-e49ab2af51a0      scaling      scaling   
4    07e2bc25-09ac-4491-bd68-b52bbbf29059    detection    detection   
..                                    ...          ...          ...   
166  f829442a-a48f-4b74-86dc-2241ddcdbe28      metrics      metrics   
167  f9aaf4e8-18e2-4f48-af22-4a044c4def8d      pixtral      pixtral   
168  fab1bd45-c87f-4267-aa29-65a4ceabb690  fine-tuning  fine tuning   
169  fe0fcf88-552b-4479-b1a1-6838d08eb490       images       images   
170  fee44b01-a2dd-4a65-bb39-51999613b207      dataset      dataset   

                                             bookmarks  
0    [{'bookmark_id': 'c6c0d547-a5de-4d68-9a48-c2f0...  
1    [{'bookmark_id': 'c23de130-b

C:\Users\pasan\AppData\Local\Temp\ipykernel_243824\2450164243.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df.groupby(["tag_id", "key", "name"]).apply(lambda g: g[[


In [2]:
validated_df = df.copy()

# Remove rows with empty title
validated_df = validated_df[validated_df["title"].notna()]

# Remove rows with empty url
validated_df = validated_df[validated_df["url"].notna()]

# Remove rows with empty tag key
validated_df = validated_df[validated_df["key"].notna()]

In [22]:
# Group by tag_id, key, and name, then count bookmarks for each tag_id
bookmark_counts = validated_df.groupby(["tag_id", "key", "name"]).size().reset_index(name="bookmark_count")

# Filter tags with more than 5 bookmarks
filtered_tags = bookmark_counts[bookmark_counts["bookmark_count"] > 5]

# print(filtered_tags)

# Filter the original dataframe to only include bookmarks with the filtered tags
filtered_df = validated_df[validated_df["tag_id"].isin(filtered_tags["tag_id"])]

# print(filtered_df)

tag_keys_df = filtered_df[["tag_id", "key"]].drop_duplicates()

print(tag_keys_df["key"].unique())



['ai' 'engineering' 'reading' 'research' 'reasoning' 'model'
 'reinforcement' 'curation' 'chatgpt' 'api' 'realtime' 'mathematics'
 'language' 'deeplearning' 'experts' 'mixtral' 'pixtral' 'optimization'
 'performance' 'programming' 'intelligence' 'coding' 'tools' 'deepseek'
 'llm' 'scaling' 'longtermism' 'mistral' 'llama' 'chat' 'fine' 'tuning'
 'gpt' 'report' 'models' 'segment' 'images' 'videos' 'speech'
 'recognition' 'supervision' 'learning' 'transferable' 'visual' 'deep'
 'large' 'scale' 'multimodal' 'shortcomings' 'exploration' 'object'
 'detection' 'unified' 'code' 'generation' 'prompt' 'segmentation'
 'sonnet' 'agents' 'computer' 'stack' 'source' 'licensing' 'memory'
 'papers' 'systems' 'implementation' 'architecture' 'huggingface'
 'benchmark' 'github' 'issues' 'benchmarks' 'system' 'hallucinations'
 'facts' 'building' 'retrieval' 'augmented' 'chatbots' 'evaluation'
 'embedding' 'datasets' 'capabilities' 'advancements' 'mteb' 'knowledge'
 'nlp' 'tasks' 'embeddings' 'context' 'po

### Clustering


In [5]:
# CLUSTERING
from topic_gen.clustering.cluster import AgglomerativeCluster, KMeansCluster


cluster = AgglomerativeCluster()
# kmeans_cluster = KMeansCluster(n_clusters=10)

# kmeans_cluster_df = kmeans_cluster.fit(filtered_df)

cluster_df = cluster.fit(tag_keys_df)
print(cluster_df)




[ 4  2 16 16  4  6  4  5 14  1  0 18  5 17  4  8  8  3  2  1  4  1  2 17
 11  3  2  8 11 14  5  3  1 15  6  0  0  0  5  0  4  4  5  0 17  5  3  0
  2  4  5  0  6  1  5 12  0 11  4  5 11 15  1  5 16  7  6  6 11 10  1  9
 10  7  8 15  6 16  2 14  2 13 10  2  2 11  4  5  7 13  5  5  3  2  9  9
 12  4 15 16 16  5 10 18 10  2  6 16  7  4  9  9]
                                    tag_id            key  cluster
0     d73bda20-81cf-4ac2-8d73-f782b68a1901             ai        4
1     23ad99a1-37e5-43dd-bd41-3ac8589cfe6f    engineering        2
2     886e8c54-87e1-4933-85d4-8520c12321cb        reading       16
3     bc9adf25-5567-4aba-abfe-3fb1f5d0e146       research       16
4     b0e3906d-3109-41a0-befa-8de4bd72b3a1      reasoning        4
...                                    ...            ...      ...
1288  ba56e98b-1fab-48a9-8471-19fc1504fba7        queries       16
1398  59d692d2-ad20-4110-bd7b-a46d0cd9ad40      multitask        7
1399  ebaad30e-c0f2-4401-a6fb-be52a6ca6c7a  understandi

In [7]:
# Print keys belonging to each cluster
for cluster_id in cluster_df["cluster"].unique():
    print(f"Cluster {cluster_id}:")
    keys = cluster_df[cluster_df["cluster"] == cluster_id]["key"].tolist()
    print(keys, len(keys))
    print()

Cluster 4:
['ai', 'reasoning', 'reinforcement', 'experts', 'intelligence', 'supervision', 'learning', 'exploration', 'agents', 'knowledge', 'training', 'understanding'] 12

Cluster 2:
['engineering', 'performance', 'tools', 'longtermism', 'shortcomings', 'augmented', 'evaluation', 'capabilities', 'advancements', 'efficiency', 'augmentation'] 11

Cluster 16:
['reading', 'research', 'papers', 'retrieval', 'questions', 'testing', 'queries'] 7

Cluster 6:
['model', 'models', 'unified', 'implementation', 'architecture', 'building', 'structure'] 7

Cluster 5:
['curation', 'language', 'fine', 'speech', 'transferable', 'large', 'object', 'generation', 'computer', 'memory', 'nlp', 'context', 'power', 'custom'] 14

Cluster 14:
['chatgpt', 'chat', 'chatbots'] 3

Cluster 1:
['api', 'programming', 'coding', 'gpt', 'code', 'licensing', 'github'] 7

Cluster 0:
['realtime', 'segment', 'images', 'videos', 'recognition', 'visual', 'multimodal', 'detection', 'segmentation'] 9

Cluster 18:
['mathematics',

In [10]:
joined_df = pd.merge(cluster_df, filtered_df, on="tag_id", how="left")
print(joined_df.head(10))

                                 tag_id key_x  cluster  \
0  d73bda20-81cf-4ac2-8d73-f782b68a1901    ai        4   
1  d73bda20-81cf-4ac2-8d73-f782b68a1901    ai        4   
2  d73bda20-81cf-4ac2-8d73-f782b68a1901    ai        4   
3  d73bda20-81cf-4ac2-8d73-f782b68a1901    ai        4   
4  d73bda20-81cf-4ac2-8d73-f782b68a1901    ai        4   
5  d73bda20-81cf-4ac2-8d73-f782b68a1901    ai        4   
6  d73bda20-81cf-4ac2-8d73-f782b68a1901    ai        4   
7  d73bda20-81cf-4ac2-8d73-f782b68a1901    ai        4   
8  d73bda20-81cf-4ac2-8d73-f782b68a1901    ai        4   
9  d73bda20-81cf-4ac2-8d73-f782b68a1901    ai        4   

                            bookmark_id  \
0  a4f37ecc-172a-48e4-8570-8d059398d77d   
1  06ba62ab-870f-432a-87a6-a5be07f7b2e6   
2  802b1856-b4cf-4b0b-bcac-b65149c9213a   
3  1785c3fe-0ec1-4208-ada2-f0366d510e1a   
4  94c217e4-d35f-442d-9d76-6a2561b15caf   
5  f6a8e377-0b53-4f09-bf43-777582c77721   
6  9006090a-94ba-4ee4-b92b-a57e2ff0f508   
7  00897913-7341-

In [11]:
cleaned_df = joined_df.drop(columns=["key_x"])
cleaned_df = cleaned_df.rename(columns={"key_y": "key"})

In [17]:
# Print keys belonging to each cluster
for cluster_id in cleaned_df["cluster"].unique():
    print(f"Cluster {cluster_id}:")
    keys_df = cleaned_df[cleaned_df["cluster"] == cluster_id]["key"]
    keys = keys_df.unique().tolist()
    bookmarks_df = cleaned_df[cleaned_df["cluster"] == cluster_id]
    bookmarks = bookmarks_df.groupby(["bookmark_id","title", "description", "language", "created_at", "updated_at"])
    print(keys, len(keys))
    print(bookmarks, len(bookmarks))
    print()

Cluster 4:
['ai', 'reasoning', 'reinforcement', 'experts', 'intelligence', 'supervision', 'learning', 'exploration', 'agents', 'knowledge', 'training', 'understanding'] 12
<pandas.core.groupby.generic.DataFrameGroupBy object at 0x00000206182F22D0> 59

Cluster 2:
['engineering', 'performance', 'tools', 'longtermism', 'shortcomings', 'augmented', 'evaluation', 'capabilities', 'advancements', 'efficiency', 'augmentation'] 11
<pandas.core.groupby.generic.DataFrameGroupBy object at 0x0000020618A8C8F0> 60

Cluster 16:
['reading', 'research', 'papers', 'retrieval', 'questions', 'testing', 'queries'] 7
<pandas.core.groupby.generic.DataFrameGroupBy object at 0x0000020618A9D9A0> 57

Cluster 6:
['model', 'models', 'unified', 'implementation', 'architecture', 'building', 'structure'] 7
<pandas.core.groupby.generic.DataFrameGroupBy object at 0x0000020618A9DEB0> 57

Cluster 5:
['curation', 'language', 'fine', 'speech', 'transferable', 'large', 'object', 'generation', 'computer', 'memory', 'nlp', 'co

In [21]:
clus_4_df = cleaned_df[cleaned_df["cluster"] == 4]

clus_tags_df = clus_4_df[["tag_id", "key"]].drop_duplicates()
clus_tags = clus_tags_df["key"].unique().tolist()

clus_bookmarks_df = clus_4_df[["bookmark_id","title", "description", "language", "created_at", "updated_at"]].drop_duplicates()
clus_titles = clus_bookmarks_df["title"].unique().tolist()

print(clus_titles, len(clus_titles))
print(clus_tags, len(clus_tags))

['The 2025 AI Engineering Reading List', 'Recent reasoning research: GRPO tweaks, base model RL, and data curation', 'Introducing ChatGPT', 'OpenAI Realtime API: The Missing Manual', 'DeepSeekMath: Pushing the Limits of Mathematical Reasoning in Open Language Models', 'Mixtral of Experts', 'Pixtral 12B', 'DeepSeek-Coder: When the Large Language Model Meets Programming -- The Rise of Code Intelligence', 'DeepSeek LLM: Scaling Open-Source Language Models with Longtermism', 'Mistral 7B', 'LLaMA: Open and Efficient Foundation Language Models', 'Llama 2: Open Foundation and Fine-Tuned Chat Models', 'GPT-4 Technical Report', 'The Llama 3 Herd of Models', 'SAM 2: Segment Anything in Images and Videos', 'Robust Speech Recognition via Large-Scale Weak Supervision', 'Eyes Wide Shut? Exploring the Visual Shortcomings of Multimodal LLMs', 'The new Claude 3.5 Sonnet, Computer Use, and Building SOTA Agents — with Erik Schluntz, Anthropic', 'Voyager: An Open-Ended Embodied Agent with Large Language M

In [ ]:
# Join the tag_keys_df with the filtered_df on tag_id
# joined_df = pd.merge(tag_keys_df, filtered_df, on="tag_id", how="left")

In [ ]:
# NOT NEEDED

# Find non-unique keys and drop rows with least bookmark count
from difflib import SequenceMatcher
import numpy as np

def similarity(a, b):
    """Calculate similarity between two strings"""
    return SequenceMatcher(None, a.lower(), b.lower()).ratio()

def find_similar_keys(df, threshold=0.8):
    """Find groups of similar keys and keep only the one with highest bookmark count"""
    
    # Get unique keys with their counts
    key_counts = df.groupby('key')['bookmark_count'].sum().reset_index()
    key_counts = key_counts.sort_values('bookmark_count', ascending=False)
    
    # Find similar key groups
    similar_groups = []
    processed_keys = set()
    
    for i, row1 in key_counts.iterrows():
        if row1['key'] in processed_keys:
            continue
            
        similar_keys = [row1['key']]
        processed_keys.add(row1['key'])
        
        for j, row2 in key_counts.iterrows():
            if i != j and row2['key'] not in processed_keys:
                if similarity(row1['key'], row2['key']) >= threshold:
                    similar_keys.append(row2['key'])
                    processed_keys.add(row2['key'])
        
        if len(similar_keys) > 1:
            similar_groups.append(similar_keys)
    
    return similar_groups, key_counts

# Find similar key groups
similar_groups, key_counts = find_similar_keys(bookmark_counts, threshold=0.8)

print("Similar key groups found:")
for i, group in enumerate(similar_groups):
    print(f"Group {i+1}: {group}")
    
    # Show counts for each key in the group
    group_counts = key_counts[key_counts['key'].isin(group)]
    print(f"Counts: {dict(zip(group_counts['key'], group_counts['bookmark_count']))}")
    print()

# Create a list of keys to keep (highest count from each similar group)
keys_to_keep = set()
keys_to_drop = set()

for group in similar_groups:
    group_counts = key_counts[key_counts['key'].isin(group)]
    # Keep the key with highest count
    best_key = group_counts.loc[group_counts['bookmark_count'].idxmax(), 'key']
    keys_to_keep.add(best_key)
    
    # Mark others for dropping
    for key in group:
        if key != best_key:
            keys_to_drop.add(key)

print(f"Keys to keep: {keys_to_keep}")
print(f"Keys to drop: {keys_to_drop}")

# Filter the dataframe to remove rows with keys that should be dropped
filtered_bookmark_counts = bookmark_counts[~bookmark_counts['key'].isin(keys_to_drop)]

print(f"\nOriginal rows: {len(bookmark_counts)}")
print(f"Filtered rows: {len(filtered_bookmark_counts)}")
print(f"Rows removed: {len(bookmark_counts) - len(filtered_bookmark_counts)}")

print("\nFiltered bookmark counts:")
print(filtered_bookmark_counts.head(10))


Similar key groups found:
Group 1: ['ai', 'api']
Counts: {'ai': 43, 'api': 25}

Group 2: ['model', 'models']
Counts: {'model': 42, 'models': 30}

Group 3: ['learning', 'deeplearning']
Counts: {'learning': 37, 'deeplearning': 27}

Group 4: ['evaluation', 'evaluations']
Counts: {'evaluation': 26, 'evaluations': 4}

Group 5: ['mathematics', 'mathematical']
Counts: {'mathematics': 25, 'mathematical': 6}

Group 6: ['benchmarks', 'benchmark', 'benchmarking']
Counts: {'benchmarks': 19, 'benchmark': 11, 'benchmarking': 5}

Group 7: ['agents', 'agent']
Counts: {'agents': 19, 'agent': 3}

Group 8: ['segmentation', 'augmentation']
Counts: {'segmentation': 17, 'augmentation': 11}

Group 9: ['embeddings', 'embedding']
Counts: {'embeddings': 16, 'embedding': 8}

Group 10: ['datasets', 'dataset']
Counts: {'datasets': 15, 'dataset': 4}

Group 11: ['videos', 'video']
Counts: {'videos': 15, 'video': 1}

Group 12: ['prompt', 'prompting']
Counts: {'prompt': 13, 'prompting': 6}

Group 13: ['systems', 'syst

In [4]:
from topic_gen.topic_gen import TopicGen
from dotenv import load_dotenv

load_dotenv()

user_id = "8c525b6b-27c9-4379-a454-2c3b3a781124"

topic_gen = TopicGen()

out_df = topic_gen.generate(user_id)
print(out_df)

[ 4  2 16 16  4  6  4  5 14  1  0 18  5 17  4  8  8  3  2  1  4  1  2 17
 11  3  2  8 11 14  5  3  1 15  6  0  0  0  5  0  4  4  5  0 17  5  3  0
  2  4  5  0  6  1  5 12  0 11  4  5 11 15  1  5 16  7  6  6 11 10  1  9
 10  7  8 15  6 16  2 14  2 13 10  2  2 11  4  5  7 13  5  5  3  2  9  9
 12  4 15 16 16  5 10 18 10  2  6 16  7  4  9  9]
ChatCompletion(id='chatcmpl-CTveAqKf7HOZfODXYSt0Za1qSiqwm', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='"Advancements in Reasoning and Learning for AI Agents"', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1761249142, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_51db84afab', usage=CompletionUsage(completion_tokens=12, prompt_tokens=1339, total_tokens=1351, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rej